In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('full_merged_dataset15.csv', encoding='utf-8')

In [3]:
df.head()

,id,title,author,snippet,link,categories,poem
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,Yours free without the asking,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']",Yours free without the asking Quick delivery v...
1,162250,All Saints&rsquo;,By Corey Van Landingham,CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin.,https://www.poetryfoundation.org/poetrymagazin...,[],CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin...
2,162279,And These Are Just a Few ...,By Melvin Dixon,This poem is for the epidemic dead and the liv...,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...",This poem is for the epidemic dead and the liv...
3,162248,ASMR,By Corey Van Landingham,Why not climb up the mountain,https://www.poetryfoundation.org/poetrymagazin...,[],Why not climb up the mountain of delight? To t...
4,162249,ASMR,By Corey Van Landingham,HelloÃ¢â‚¬â€Tonight,https://www.poetryfoundation.org/poetrymagazin...,[],HelloÃ¢â‚¬â€Tonight weÃ¢â‚¬â„¢ll trace the st...


In [4]:
# i want to remove the rows where the value of the column 'label' is 'unknown'
df2 = df.iloc[:, :7]


In [5]:
df2.head()

,id,title,author,snippet,link,categories,poem
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,Yours free without the asking,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']",Yours free without the asking Quick delivery v...
1,162250,All Saints&rsquo;,By Corey Van Landingham,CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin.,https://www.poetryfoundation.org/poetrymagazin...,[],CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin...
2,162279,And These Are Just a Few ...,By Melvin Dixon,This poem is for the epidemic dead and the liv...,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...",This poem is for the epidemic dead and the liv...
3,162248,ASMR,By Corey Van Landingham,Why not climb up the mountain,https://www.poetryfoundation.org/poetrymagazin...,[],Why not climb up the mountain of delight? To t...
4,162249,ASMR,By Corey Van Landingham,HelloÃ¢â‚¬â€Tonight,https://www.poetryfoundation.org/poetrymagazin...,[],HelloÃ¢â‚¬â€Tonight weÃ¢â‚¬â„¢ll trace the st...


In [6]:
# df2.to_csv('full_merged_dataset15.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used


In [7]:
import re
import nltk

def check_nltk_packages():
  packages = ['punkt','stopwords','omw-1.4','wordnet']

  for package in packages:
    try:
      nltk.data.find('tokenizers/' + package)
    except LookupError:
      nltk.download(package)
check_nltk_packages()

try:
  import lxml
except ModuleNotFoundError:
  %pip install lxml

try:
  import contractions
except ModuleNotFoundError:
  %pip install contractions
  import contractions

from bs4 import BeautifulSoup
import re

[nltk_data] Downloading package stopwords to /home/vm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vm/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Pre-compile the patterns for efficiency
url_pattern = re.compile(r'https://\S+|www\.\S+')
unwanted_chars_pattern = re.compile(r'Ã¢â‚¬Å|Ã¢â‚¬â„¢|Ã¢â‚¬â€|â‚¬Âº|ÃƒÂ¢Ã¢â€šÂ¬Ã|Ã|¬|¢|š|€||âº|â|Âº|Ã¢|Ãƒâ€|¬ÂºÃƒâ€¦Ã¢â|¬|Å|¾|Âº|Ã¢|Âº|Ã¢â‚¬ÂºÃƒâ€')

def wrangle_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "lxml").text
    
    # Remove URLs
    text = url_pattern.sub('', text)
    
    # Fix specific character issues in one go
    text = unwanted_chars_pattern.sub('', text)

    # Expand contractions once, after all substitutions
    text = contractions.fix(text)
    
    return text

In [9]:


poem = df2['poem'].iloc[1] # This is the first review
wrangled_poem = wrangle_text(poem)
print(colored('\n============= First poem in corpus =============', 'blue'))
print(poem)
print(colored('\n============= After wrangling result =============', 'green'))
print(wrangled_poem)


============= First poem in corpus =============
CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin. Bulbs rotting in the damp garageÃ¢â‚¬â€still no hard frost. Lamplight, low moon. ItÃ¢â‚¬â„¢s that time when corners deepen and the frame wonÃ¢â‚¬â„¢t stay untilted. A tendency toward extremes, one doctor diagnosed. ItÃ¢â‚¬â„¢s just that out the window stands a row of white globes too long now. Too long. Why do you keep looking? When the child asks, of the man in the moon, who cut off his head, who says turn away, turn away from that. A mystery of faith, as it were, that the light indeed returns.

============= After wrangling result =============
Caravaggios face in the sunken pumpkin. Bulbs rotting in the damp garagestill no hard frost. Lamplight, low moon. Its that time when corners deepen and the frame will not stay untilted. A tendency toward extremes, one doctor diagnosed. Its just that out the window stands a row of white globes too long now. Too long. Why do you keep looking? When 

In [10]:
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize


print(colored('\n============= First poem in data =============', 'blue'))
print(wrangled_poem)

# Tokenize the review
poem_tokens = wordpunct_tokenize(wrangled_poem)
# Tokenize the review at the sentence level
poem_tokens_sent = sent_tokenize(wrangled_poem)

print(colored('\n============= First poem (tokens) =============', 'green'))
print(poem_tokens)

print(colored('\n============= First poem (tokens sent level) =============', 'red'))
print(poem_tokens_sent)


============= First poem in data =============
Caravaggios face in the sunken pumpkin. Bulbs rotting in the damp garagestill no hard frost. Lamplight, low moon. Its that time when corners deepen and the frame will not stay untilted. A tendency toward extremes, one doctor diagnosed. Its just that out the window stands a row of white globes too long now. Too long. Why do you keep looking? When the child asks, of the man in the moon, who cut off his head, who says turn away, turn away from that. A mystery of faith, as it were, that the light indeed returns.

============= First poem (tokens) =============
['Caravaggios', 'face', 'in', 'the', 'sunken', 'pumpkin', '.', 'Bulbs', 'rotting', 'in', 'the', 'damp', 'garagestill', 'no', 'hard', 'frost', '.', 'Lamplight', ',', 'low', 'moon', '.', 'Its', 'that', 'time', 'when', 'corners', 'deepen', 'and', 'the', 'frame', 'will', 'not', 'stay', 'untilted', '.', 'A', 'tendency', 'toward', 'extremes', ',', 'one', 'doctor', 'diagnosed', '.', 'Its', 'ju

### Converting into lower case & removing alphanumeric: 

In [11]:
poem_tokens_filtered = [token.lower() for token in poem_tokens if token.isalnum()]

# The isalnum() method returns True if all characters in the string are 
# alphanumeric (either alphabets or numbers). If not, it returns False.

print(colored('\n============= First poem (tokens) =============', 'blue'))
print(poem_tokens)

print(colored('\n============= First poem (tokens filtered) =============', 'green'))
print(poem_tokens_filtered)


============= First poem (tokens) =============
['Caravaggios', 'face', 'in', 'the', 'sunken', 'pumpkin', '.', 'Bulbs', 'rotting', 'in', 'the', 'damp', 'garagestill', 'no', 'hard', 'frost', '.', 'Lamplight', ',', 'low', 'moon', '.', 'Its', 'that', 'time', 'when', 'corners', 'deepen', 'and', 'the', 'frame', 'will', 'not', 'stay', 'untilted', '.', 'A', 'tendency', 'toward', 'extremes', ',', 'one', 'doctor', 'diagnosed', '.', 'Its', 'just', 'that', 'out', 'the', 'window', 'stands', 'a', 'row', 'of', 'white', 'globes', 'too', 'long', 'now', '.', 'Too', 'long', '.', 'Why', 'do', 'you', 'keep', 'looking', '?', 'When', 'the', 'child', 'asks', ',', 'of', 'the', 'man', 'in', 'the', 'moon', ',', 'who', 'cut', 'off', 'his', 'head', ',', 'who', 'says', 'turn', 'away', ',', 'turn', 'away', 'from', 'that', '.', 'A', 'mystery', 'of', 'faith', ',', 'as', 'it', 'were', ',', 'that', 'the', 'light', 'indeed', 'returns', '.']

============= First poem (tokens filtered) =============
['caravaggios', 'face

In [12]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

stemmer = SnowballStemmer('english')
wnl = WordNetLemmatizer()

stemmed_poem = [stemmer.stem(el) for el in poem_tokens_filtered]
print(colored('\n============= Stemmed review  =============', 'blue'))
print(stemmed_poem)

lemmatized_poem = [wnl.lemmatize(el) for el in poem_tokens_filtered]
print(colored('\n============= Lemmatized review  =============', 'blue'))
print(lemmatized_poem)

# The difference between both is that the lemmatization process is more complex
# and it involves resolving words to their base or root form.

# Lemmatization: Lemmatization is the process of converting a word to its base form.
# Stemming: Stemming is the process of reducing a word to its root form.

# For example: 
# Lemmatization: The word "better" is converted to "good".
# Stemming: The word "better" is converted to "bet".


============= Stemmed review  =============
['caravaggio', 'face', 'in', 'the', 'sunken', 'pumpkin', 'bulb', 'rot', 'in', 'the', 'damp', 'garagestil', 'no', 'hard', 'frost', 'lamplight', 'low', 'moon', 'it', 'that', 'time', 'when', 'corner', 'deepen', 'and', 'the', 'frame', 'will', 'not', 'stay', 'untilt', 'a', 'tendenc', 'toward', 'extrem', 'one', 'doctor', 'diagnos', 'it', 'just', 'that', 'out', 'the', 'window', 'stand', 'a', 'row', 'of', 'white', 'globe', 'too', 'long', 'now', 'too', 'long', 'whi', 'do', 'you', 'keep', 'look', 'when', 'the', 'child', 'ask', 'of', 'the', 'man', 'in', 'the', 'moon', 'who', 'cut', 'off', 'his', 'head', 'who', 'say', 'turn', 'away', 'turn', 'away', 'from', 'that', 'a', 'mysteri', 'of', 'faith', 'as', 'it', 'were', 'that', 'the', 'light', 'inde', 'return']

============= Lemmatized review  =============
['caravaggio', 'face', 'in', 'the', 'sunken', 'pumpkin', 'bulb', 'rotting', 'in', 'the', 'damp', 'garagestill', 'no', 'hard', 'frost', 'lamplight', 'low

In [13]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

In [14]:
# Initialize NLTK resources
stopwords_en = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

# Pre-compile the patterns for efficiency
url_pattern = re.compile(r'https://\S+|www\.\S+')
unwanted_chars_pattern = re.compile(r'Ã¢â‚¬Å|Ã¢â‚¬â„¢|Ã¢â‚¬â€|â‚¬Âº|ÃƒÂ¢Ã¢â€šÂ¬Ã|Ã|¬|¢|š|€||âº|â|Âº|Ã¢|Ãƒâ€|¬ÂºÃƒâ€¦Ã¢â|¬|Å|¾|Âº|Ã¢|Âº|Ã¢â‚¬ÂºÃƒâ€')

def wrangle_text(text):
    # Check if text is a string
    if isinstance(text, str):
        # Remove HTML tags
        text = BeautifulSoup(text, "lxml").text

        # Remove URLs
        text = url_pattern.sub('', text)

        # Fix specific character issues in one go
        text = unwanted_chars_pattern.sub('', text)

        # Expand contractions once, after all substitutions
        text = contractions.fix(text)

    return text

def prepare_data(text, tool='lemmatize'):
    # Wrangle text
    text = wrangle_text(text)

    # Tokenize text
    tokens = wordpunct_tokenize(text)

    # Filter tokens
    tokens_filtered = [token.lower() for token in tokens if token.isalnum()]

    if tool == 'stem':
        # Stem tokens
        tokens_filtered = [stemmer.stem(el) for el in tokens_filtered]
    elif tool == 'lemmatize':
        # Lemmatize tokens
        tokens_filtered = [wnl.lemmatize(el) for el in tokens_filtered]
    else:
        raise ValueError('Invalid tool. Please select either "stem" or "lemmatize".')

    # Remove stopwords
    clean_tokens = [el for el in tokens_filtered if el not in stopwords_en]

    return clean_tokens

In [15]:
# Now lets use the original text and check it: 
print(colored('\n============= First review in corpus =============', 'blue'))
print(poem)

# Prepare the data
clean_tokens = prepare_data(poem, tool = "lemmatize")
print(colored('\n============= Clean tokens =============', 'green'))
print(clean_tokens)



============= First review in corpus =============
CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin. Bulbs rotting in the damp garageÃ¢â‚¬â€still no hard frost. Lamplight, low moon. ItÃ¢â‚¬â„¢s that time when corners deepen and the frame wonÃ¢â‚¬â„¢t stay untilted. A tendency toward extremes, one doctor diagnosed. ItÃ¢â‚¬â„¢s just that out the window stands a row of white globes too long now. Too long. Why do you keep looking? When the child asks, of the man in the moon, who cut off his head, who says turn away, turn away from that. A mystery of faith, as it were, that the light indeed returns.

============= Clean tokens =============
['caravaggio', 'face', 'sunken', 'pumpkin', 'bulb', 'rotting', 'damp', 'garagestill', 'hard', 'frost', 'lamplight', 'low', 'moon', 'time', 'corner', 'deepen', 'frame', 'stay', 'untilted', 'tendency', 'toward', 'extreme', 'one', 'doctor', 'diagnosed', 'window', 'stand', 'row', 'white', 'globe', 'long', 'long', 'keep', 'looking', 'child', 'asks', 'man', 'm

In [16]:
import numpy as np

# Drop rows with missing values in the 'poem' column
df2 = df2.dropna(subset=['poem'])

# Apply the prepare_data function to clean the poems
df2['clean_tokens'] = df2['poem'].apply(lambda x: prepare_data(x, tool='lemmatize'))


/tmp/ipykernel_4854/1307946892.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [17]:
df2['clean_tokens'].head(10)

0    [free, without, asking, quick, delivery, via, ...
1    [caravaggio, face, sunken, pumpkin, bulb, rott...
2    [poem, epidemic, dead, living, remember, neigh...
3    [climb, mountain, delight, world, thin, meridi...
4    [hellotonight, well, trace, static, bough, tem...
5    [didier, vermont, leaf, glass, hand, working, ...
6    [say, land, remembered, muddy, rush, slows, ne...
7    [galleria, nazionale, darte, antica, rome, wa,...
8    [net, headfirst, prairie, spring, seeing, noth...
9    [kara, walker, blue, â, 2020, everyday, brown,...
Name: clean_tokens, dtype: object

### Snipet column preprocessing

In [18]:
df2 = df2.dropna(subset=['snippet'])

# Apply the prepare_data function to clean the poems
df2['snippet'] = df2['snippet'].apply(lambda x: prepare_data(x, tool='lemmatize'))

/tmp/ipykernel_4854/1307946892.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [ ]:
df2['snippet'].head(10)

In [19]:
# remove the column 'poem'
df2 = df2.drop(columns=['poem'])

In [21]:
df2.to_csv('final_project_data_2.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used


In [22]:


df2.head(50)

,id,title,author,snippet,link,categories,clean_tokens
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,"[free, without, asking]",https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']","[free, without, asking, quick, delivery, via, ..."
1,162250,All Saints&rsquo;,By Corey Van Landingham,"[caravaggio, face, sunken, pumpkin]",https://www.poetryfoundation.org/poetrymagazin...,[],"[caravaggio, face, sunken, pumpkin, bulb, rott..."
2,162279,And These Are Just a Few ...,By Melvin Dixon,"[poem, epidemic, dead, living, remember]",https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...","[poem, epidemic, dead, living, remember, neigh..."
3,162248,ASMR,By Corey Van Landingham,"[climb, mountain]",https://www.poetryfoundation.org/poetrymagazin...,[],"[climb, mountain, delight, world, thin, meridi..."
4,162249,ASMR,By Corey Van Landingham,[hellotonight],https://www.poetryfoundation.org/poetrymagazin...,[],"[hellotonight, well, trace, static, bough, tem..."
5,162276,Autumn Leaving,By Melvin Dixon,"[vermont, leaf, glass]",https://www.poetryfoundation.org/poetrymagazin...,[],"[didier, vermont, leaf, glass, hand, working, ..."
6,162292,Babble,By Nica Giromini,"[say, land]",https://www.poetryfoundation.org/poetrymagazin...,[],"[say, land, remembered, muddy, rush, slows, ne..."
7,162295,Chiaroscuro Spring time,By Cyrus Cassells,"[wa, season, think, write, indoors]",https://www.poetryfoundation.org/poetrymagazin...,"['Arts & Sciences', 'Painting & Sculpture']","[galleria, nazionale, darte, antica, rome, wa,..."
8,162291,Currency,By Nica Giromini,"[net, headfirst]",https://www.poetryfoundation.org/poetrymagazin...,[],"[net, headfirst, prairie, spring, seeing, noth..."
9,162267,Everyday around the world a woman is pulled in...,By Krista Franklin,"[everyday, brown]",https://www.poetryfoundation.org/poetrymagazin...,"['Types/Modes', 'Ekphrasis']","[kara, walker, blue, â, 2020, everyday, brown,..."
